In [1]:
from sklearn.ensemble import RandomForestRegressor
import joblib
import numpy as np
from sklearn.metrics import accuracy_score
from quadratic_weighted_kappa import quadratic_weighted_kappa
import pandas as pd

In [2]:
x = joblib.load('essay_ease10_sbert768_simbow_langerr_780_normalized_asap6')
asap = joblib.load("asap6_new_adjudication")
x.shape

(1800, 780)

In [3]:
asap.head()

,essay,rater1_domain1,rater2_domain1,domain1_score,higher,mean,sum,check
0,There were many obstacles that the builders fa...,2,2,2,2,2.0,4,True
1,"Him from the start, there would have been many...",3,3,3,3,3.0,6,True
2,The builders of the Empire State Building face...,3,4,4,4,4.0,7,True
3,In the passage The Mooring Mast by Marcia Amid...,1,1,1,1,1.0,2,True
4,The builders of the Empire State Building face...,3,3,3,3,3.0,6,True


In [24]:
y = asap["sum"]

In [25]:
def get_feature_names_extended():
    ease_feats = ['Answer Length', 'Word Counts', 'Average Word Length', 'Good n-gram', 'Prompt Overlap',
              'Prompt Overlap (synonyms)', 'Punctuation Counts', 'Spelling Error', 'Unique Words', 'Prompt Similarity SBert']

    sbert_feats = []
    sbert_dim = 768

    for i in range(0, sbert_dim):
    	fname = "sbert_" + str(i) 
    	sbert_feats.append(fname)
    
    prompt_similarity_bow = ["Prompt Similarity BOW"]
    lang_error = ["Language Error"]
    
    feature_names = ease_feats + prompt_similarity_bow + lang_error + sbert_feats 

    print("len feature names: ", len(feature_names))
    
    return feature_names

feature_names = get_feature_names_extended()[:12]

len feature names:  780


In [26]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

print(kf)

KFold(n_splits=5, random_state=42, shuffle=True)


In [27]:
model2 = RandomForestRegressor(n_estimators = 100)

In [28]:
qwk_scores = []

test_indices = []
pred_labels = []

for train_index, test_index in kf.split(x, y):
#     print("train_index: ", len(train_index))
#     print("test_index: ", len(test_index))
#     print("train_index: ", train_index[:20])
#     print("test_index: ", test_index[:20])
    
    X_train, X_test, Y_train, Y_test = x[train_index], x[test_index], y[train_index], y[test_index]

    #d_train = xgboost.DMatrix(X_train, label=Y_train, feature_names=feature_names)
    #d_test = xgboost.DMatrix(X_test, label=Y_test, feature_names=feature_names)
         
    model2.fit(X_train, Y_train)
    
    #rf.fit(X_train, Y_train)
    
    #predict = model.predict(d_test)
    predict = model2.predict(X_test)
    #predict = rf.predict(X_test)
    predict = np.round(predict)
    
    pred_labels.extend(predict)
    test_indices.extend(test_index)

    result = quadratic_weighted_kappa(Y_test, predict)
    
    print("Qwk : ", result)
    
    qwk_scores.append(result)
    
    
np.mean(qwk_scores)

Qwk :  0.8234375
Qwk :  0.8044380061895042
Qwk :  0.8190664808430524
Qwk :  0.831701023777155
Qwk :  0.8402733313318792


0.8237832684283182

In [29]:
print(len(pred_labels))
print(len(test_indices))

1800
1800


In [30]:
pred_labels_int = list(map(int, pred_labels))

In [31]:
new_score = np.zeros(len(pred_labels))
new_score

array([0., 0., 0., ..., 0., 0., 0.])

In [32]:
new_score[test_indices] = pred_labels_int

In [33]:
joblib.dump(new_score, 'score_model_normalized_asap6_rf_sum')

['score_model_normalized_asap6_rf_sum']